## Import Libraries

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import glob
import librosa
import librosa.display as ld
# Printa todos os valores do array
#np.set_printoptions(threshold=np.inf)
np.set_printoptions(threshold=10)
import pandas as pd
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
import keras
import os
import re
from sklearn.externals import joblib

/home/fernanda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### To load records

In [3]:
#data = glob.glob('1seg/test/*.mp3')
#data = glob.glob('2seg/fold1/*.mp3')
#data = glob.glob('../../Projeto_phd/academia/*.wav')
data_casa = glob.glob('casa/*.wav')
data_academia = glob.glob('academia/*.wav')
#data_faculdade = glob.glob('faculdade/*.wav')
#print ((data_casa))

In [4]:
def load_datas(data):
    len_audios = []
    audios = []

    for file in sorted(data):
        audio, sr = librosa.core.load(file, sr=None, mono=True, offset=0.0, duration=None)
        len_audios.append(len(audio))
        audios.append(audio)
        max_audio = max(len_audios)
        
    return audios, max(len_audios), sr

def spectrograms(audios, len_audios, sr):
    
    hop_length = 400
    n_fft = 500

    #To convert the hop length and frame size to units of seconds:
    #print ("hop length[s]:", float(hop_length)/sr) # units of seconds
    #print ("frame size[s]:",float(n_fft)/sr) # units of seconds

    audios_resize = []
    specs = []

    for i in range(len(audios)):
        shape = (len_audios)
        i_audio = np.array(audios[i])
        i_audio.resize(shape)
        x = librosa.stft(i_audio, n_fft=n_fft, hop_length=hop_length)
        X = librosa.amplitude_to_db(np.abs(x), ref=np.max)
        specs.append(X)
        audios_resize.append(i_audio)
        
    return specs

In [7]:
audios_a, len_audios_a, sr = load_datas(data_academia)
audios_c, len_audios_c, sr = load_datas(data_casa)
#audios_f, len_audios_f, sr = load_datas(data_faculdade)

print(len_audios_a, len_audios_c)

def spectro_length(audio1, len1, audio2, len2, sr):
    
    if len1 > len2:
        specs1 = spectrograms(audio1, len1, sr)
        specs2 = spectrograms(audio2, len1, sr)
    else:
        specs1 = spectrograms(audio1, len2, sr)
        specs2 = spectrograms(audio2, len2, sr)
    return specs1, specs2

specs_a, specs_c = spectro_length(audios_a, len_audios_a, audios_c, len_audios_c, sr)

print (specs_a[0].shape)
print (specs_c[0].shape)

67356 56905
(251, 169)
(251, 169)


#### To load Labels

In [22]:
def labeling(pathname):
    l = []
    for file in pathname:
        (dirname, sname) = os.path.split(file)
        (name, ext) = os.path.splitext(sname)
        #print (name)
        n = re.split('[0-9]+', name, flags=re.IGNORECASE)
        l.append([n[0]])
    return l

#l_casa = np.array(labeling(data_casa))
l_casa = labeling(data_casa)
l_academia = labeling(data_academia)

#type(l_casa)

In [47]:
x_train, x_test1, y_train, y_test1 = train_test_split(specs_c, l_casa, test_size=0.5, random_state=42, stratify=l_casa)
x_test, x_val, y_test, y_val = train_test_split(x_test1, y_test1, test_size=0.5, random_state=42, stratify=y_train1)
    

In [48]:
len(x_train), len(x_test), len(x_val)

(200, 100, 100)

## To separate 80% to train and 20% to test

In [27]:
#stratify: certifica a mesma quantidade de audios de cada classe.

def holdout(data1, l1, data2, l2):
    
    x_train1, x_test, y_train1, y_test = train_test_split(data1, l1, test_size=0.5, random_state=42, stratify=l1)
    x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, test_size=0.8, random_state=42, stratify=y_train1)
    
    X_train1, X_test, Y_train1, Y_test = train_test_split(data2, l2, test_size=0.5, random_state=42, stratify=l2)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train1, Y_train1, test_size=0.8, random_state=42, stratify=Y_train1)
    
    #x_tr = np.concatenate(x_train, X_train)
    #x_v = np.concatenate(x_val, X_val)
    #x_te = np.concatenate(x_test, X_test)
    
    x_tr = x_train + X_train
    x_v = x_val + X_val
    x_te = x_test + X_test
    
    y_tr = y_train + Y_train
    y_v = y_val + Y_val
    y_te = y_test + Y_test
    
    return x_tr, x_v, x_te, y_tr, y_v, y_te

In [28]:
x_train, x_val, x_test, y_train, y_val, y_test = holdout(specs_a, l_academia, specs_c,l_casa)

In [34]:
len(x_train), len(x_val), len(x_test)

(80, 320, 400)

In [ ]:
#print (type(y_train))
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)
y_val_np = np.array(y_val)
x_val_np = np.array(x_val)
#print (y_train_np)

In [ ]:
joblib.dump(x_train_np, "features_propaganda/2segs/x_train.pkl")
joblib.dump(x_test_np, "features_propaganda/2segs/x_test.pkl")
joblib.dump(x_val_np, "features_propaganda/2segs/x_val.pkl")
joblib.dump(y_train_np, "features_propaganda/2segs/y_train.pkl")
joblib.dump(y_test_np, "features_propaganda/2segs/y_test.pkl")
joblib.dump(y_val_np, "features_propaganda/2segs/y_val.pkl")
